In [1]:
from temp_profile import TempProfile2D
from eutectic_interface import EuteticInterface

import glob
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import itertools
\
from scipy.stats import pearsonr
from scipy.signal import find_peaks, peak_prominences

import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
twod_paths = glob.glob('../data/gaia/temp_profile/europa/EuropaParameterStudy/*_2D_data.txt')
twod_dict={}


for filepath in twod_paths:
    temp_profile = TempProfile2D.from_filepath(filepath)
    twod_dict[filepath] = temp_profile

In [3]:
eutectics_stats = pd.read_csv("../data/gaia/temp_profile/europa/EuropaParameterStudy/cell_counting/master_eutectic_df.csv")
eutectics_stats.head()

eutectics_stats = eutectics_stats.loc[:, 'Total Shell Depth [Km]':'Cells Per Kilometer']
eutectics_stats['Viscosity [Pa s]'] = eutectics_stats['Viscosity [Pa s]'].str.replace('eta1e+', '').astype(int)
eutectics_stats.sort_values(by = ['Total Shell Depth [Km]', 'Viscosity [Pa s]'])

,Total Shell Depth [Km],Viscosity [Pa s],Deapest Downwelling [Km],Mean Upwelling Height [Km],Convective Height Imbalance Ratio,Estimated Number of Convection Cells,Cells Per Kilometer
1,25.0,12,10.238,5.790242,1.758109,165,0.097744
10,25.0,13,10.394,6.317776,2.110294,85,0.050353
4,25.0,14,14.298,9.210346,3.870923,52,0.030804
14,25.0,15,20.069,0.000000,0.000000,0,0.000000
11,30.0,14,17.339,10.453604,4.440521,48,0.028435
0,30.0,15,24.149,0.000000,0.000000,0,0.000000
15,40.0,14,20.667,12.250351,5.142973,37,0.021918
5,40.0,15,26.762,19.921196,5.468196,46,0.027250
7,60.0,12,12.111,5.342018,1.800566,113,0.066940
12,60.0,13,12.111,7.468431,2.669985,65,0.038505


In [4]:
eutectics_stats.columns

Index(['Total Shell Depth [Km]', 'Viscosity [Pa s]',
       'Deapest Downwelling [Km]', 'Mean Upwelling Height [Km]',
       'Convective Height Imbalance Ratio',
       'Estimated Number of Convection Cells', 'Cells Per Kilometer'],
      dtype='object')

In [5]:
def corrfunc(x, y, **kws):
    r, p = pearsonr(x, y)
    ax = plt.gca()
    ax.annotate(f'R={r:.2f}\nP={p:.2g}', 
                xy=(0.1, 0.9), xycoords=ax.transAxes,
                fontsize=12, ha='left', va='center')

g = sns.pairplot(
    eutectics_stats,
    y_vars=["Total Shell Depth [Km]", "Viscosity [Pa s]"],
    x_vars = ['Deapest Downwelling [Km]','Mean Upwelling Height [Km]', 'Estimated Number of Convection Cells','Convective Height Imbalance Ratio', 'Cells Per Kilometer'],
    plot_kws={'line_kws':{'color':'red'}},
    aspect = 1.3,
    kind = 'reg'
)

g.map(corrfunc)

/Users/willbyrne/opt/anaconda3/envs/dlr_radar/lib/python3.10/site-packages/seaborn/axisgrid.py:123: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)


In [ ]:
eutectics_stats.columns[2:]

#list(itertools.combinations())

Index(['Deapest Downwelling [Km]', 'Mean Upwelling Height [Km]',
       'Convective Height Imbalance Ratio',
       'Estimated Number of Convection Cells', 'Cells Per Kilometer'],
      dtype='object')

In [ ]:


# Create a list to store the regression models
regression_models = []
# Define the column combinations



column_combinations = list(itertools.combinations(eutectics_stats.columns, 2))

# Iterate over the column combinations
for combination in column_combinations:
    # Extract the column names
    x1, x2 = combination
    
    # Create the feature matrix X
    X = eutectics_stats[[x1, x2]]
    
    # Create the target variable y
    y = eutectics_stats['Viscosity [Pa s]']
    
    # Create a linear regression model
    model = LinearRegression()
    
    # Fit the model
    model.fit(X, y)
    
    # Append the model to the list
    regression_models.append(model)
    
    # Calculate the coefficient of determination (R-squared)
    r_squared = model.score(X, y)
    
    # Print the performance of the model
    print(f"Model for {x1} and {x2}: R-squared = {r_squared}")


Model for Total Shell Depth [Km] and Viscosity [Pa s]: R-squared = 1.0
Model for Total Shell Depth [Km] and Deapest Downwelling [Km]: R-squared = 0.2863280597390606
Model for Total Shell Depth [Km] and Mean Upwelling Height [Km]: R-squared = 0.139080312437162
Model for Total Shell Depth [Km] and Convective Height Imbalance Ratio: R-squared = 0.13422055527031562
Model for Total Shell Depth [Km] and Estimated Number of Convection Cells: R-squared = 0.7708007110541818
Model for Total Shell Depth [Km] and Cells Per Kilometer: R-squared = 0.7708007110541819
Model for Viscosity [Pa s] and Deapest Downwelling [Km]: R-squared = 1.0
Model for Viscosity [Pa s] and Mean Upwelling Height [Km]: R-squared = 1.0
Model for Viscosity [Pa s] and Convective Height Imbalance Ratio: R-squared = 1.0
Model for Viscosity [Pa s] and Estimated Number of Convection Cells: R-squared = 1.0
Model for Viscosity [Pa s] and Cells Per Kilometer: R-squared = 1.0
Model for Deapest Downwelling [Km] and Mean Upwelling Heig